<div class="alert alert-block alert-success"><font color="black"><h1><left>Scrub MIMIC ICU Data for Intestinal Conditions</left></h1></font></div>

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
path = 'E:\Education\DataScience\CCSU-Data-Science\Data-Mining-And-Predictive-Analytics\Data-532\Data\\'
file1 = path + 'pd_notes.snappy.parquet'
file2 = path + 'discharge_notes.snappy.parquet'

In [3]:
# How many notes from each file
pn_notes = pd.read_parquet(file1,engine='auto',use_nullable_dtypes=True)
print('admissions: ',len(pn_notes['hadm_id'].unique()))
discharge_notes = pd.read_parquet(file2,engine='auto',use_nullable_dtypes=True)
print('discharge-notes: ',len(discharge_notes['hadm_id'].unique()))

admissions:  2694
discharge-notes:  6274


In [4]:
# print 5 notes and their length to guarantee format before processing
pn_notes.columns 
note_len = [len(x) for x in pn_notes['text']]
print(note_len[1:5])
pn_notes.iloc[[1,2,3,4,5],[17]]

[22554, 22210, 16203, 30477]


,text
1,Ms [**Known lastname 406**] is a 24 yo F with ...
2,"24YOF w/ ESRD, HTN started on OD this week. Pr..."
3,Admitted to SICU under MICU service at 0620 th...
4,24yo female pt with h/o ESRD on regular HD ( t...
5,NURSING MICU NOTE 7P-7A PT [**Name (NI) 577**...


In [5]:
# note counts by admission type
pn_notes.groupby('admission_type')['hadm_id'].count()


admission_type
EMERGENCY    2624
URGENT         70
Name: hadm_id, dtype: int64

In [6]:
def store_med_tags(df):
    
    import operator

    tags_dict = {}
    for row in df.itertuples():
        tags = findall_med_tags(row.text)
        if tags is not None:
            for tag in tags:
                temp_tag = str(tag)
#                if len(tags) > 0 :
#                    print('tag: ',tags)
                if temp_tag in tags_dict.keys():
                    tags_dict[temp_tag] += 1
                else:
                    tags_dict[temp_tag] = 1
    
    sorted_d = dict( sorted(tags_dict.items(), key=operator.itemgetter(1),reverse=True))
#    print('Dictionary in descending order by value : ',len(sorted_d))
     
    return sorted_d
#    return sorted_d


def findall_med_tags(note_data):

# https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
    #print('in data length: ', len(note_data))
    
    regex2 = r"(\[(.+?)\])"
    
    note_data = re.sub(r'\[.*?:.*?\]', ' ', note_data)
    note_data = re.sub(r'\[\*\*.*?\*\*\]', ' ', note_data)
    note_data = re.sub(r'\s{2,99}', ' ', note_data)
    note_data = re.sub(r'\n', ' ', note_data)
#   note_data = re.sub(regex, ' ', note_data)
    note_data = note_data.upper()
    tags= re.findall(regex2, note_data)
    
#    if len(tags) > 0 :
#        print('tag length: ', len(tags))
#        print('tags:\n ', tags)
        
    return tags


def write_dict_csv(outfile, adict):

    
    import csv
    print('Write input Dictionary: ',len(adict.keys()))
    field_names = ['word','count']
    
    

    with open(outfile, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()
        for key in adict:
#            if adict[key] > 300:
            writer.writerow({'word': key, 'count': adict[key]})
    csvfile.close()   
    return
    
file2 = path + 'big_notes_tags_4-7-23.csv'
adict = store_med_tags(pn_notes)
if adict is not None:
    write_dict_csv(file2, adict)


Write input Dictionary:  102


In [7]:
# Routine to remove unneccessary characters
import re

def preprocess1(x):
    
#    print('in2a: ', 'x length ', len(x),  '', x[0:20])
    y = re.sub('\\[(.*?)\\]', '', x)
#    print('in2b: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('[0-9]+\.', '', y)  # remove 1.2. since the segmenter segments based on this
#    print('in2c: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('dr\.', 'doctor', y)
#    print('in2d: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('m\.d\.', 'md', y)
#    print('in2e: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('admission date:', '', y)
#    print('in2f: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('discharge date:', '', y)
#    print('in2g: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('--|__|==', ' ', y)
    
     # remove, digits, spaces
#    print('in2h: ', 'y length ', len(y),  ' ', y[0:20])
    y = y.translate(str.maketrans("", "", string.digits))
#    print('in2i: ', 'y length ', len(y),  ' ', y[0:20])
    y = " ".join(y.split())
#    print('in2j: ', 'y length ', len(y),  ' ', y[0:20])
    
    return y

In [8]:
# function to process each note, call above function, get rid of line feeds, and make lowercase
from tqdm import tqdm, trange 
def preprocessing(df_less_n):
#    print('in2: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].fillna(' ')
#    print('in3: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.replace('\n', ' ')
#    print('in4: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.replace('\r', ' ')
#    print('in5: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].apply(str.strip)
#    print('in6: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.lower()
#    print('in7: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].apply(lambda x: preprocess1(x))
#    print('in8: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    return df_less_n



In [9]:
# run preprocess for discharge and adm notes
import string
discharge_notes = preprocessing(discharge_notes)
pn_notes = preprocessing(pn_notes)
# print first 2 texts columns in each dataset as a sanity check after cleanup
print(pn_notes[0:2].text[0:10])
print(discharge_notes[0:2].text[0:10])

0    nursing a-p see adm hx for pmh & events leadin...
1    ms is a yo f with lupus since age , with esrd ...
Name: text, dtype: object
0    date of birth: sex: m service: medicine allerg...
1    date of birth: sex: m service: cardiothoracic ...
Name: text, dtype: object


In [10]:
path = 'E://Education//DataScience/CCSU-Data-Science/Data-Mining-And-Predictive-Analytics/Data-532/Data/'
pn_notes.to_parquet(path + 'pd_notes_clean.snappy.parquet',compression='snappy',index=None)  
discharge_notes.to_parquet(path + 'discharge_notes_clean.snappy.parquet',compression='snappy',index=None)  

In [11]:
import boto3, os
s3 = boto3.resource('s3')
path = 'E://Education//DataScience/CCSU-Data-Science/Data-Mining-And-Predictive-Analytics/Data-532/Data/'
f1name = 'pd_notes_clean.snappy.parquet'
f2name = 'discharge_notes_clean.snappy.parquet'

In [12]:
subdir = 'MIMIC/Data/' 
S3_path = subdir + f1name
out_file = path + f1name
s3.meta.client.upload_file(out_file, 'depratti-masters-thesis', S3_path)
S3_path = subdir + f2name
out_file = path + f2name
s3.meta.client.upload_file(out_file, 'depratti-masters-thesis', S3_path)

In [13]:
print('pn_notes shape: ', pn_notes.shape)
print('pn_notes columns: ', pn_notes.columns)

pn_notes shape:  (2694, 19)
pn_notes columns:  Index(['subject_id', 'hadm_id', 'icd9_codes', 'primary_icd9_code',
       'primary_icd10_code', 'stay_hours', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group', 'text',
       'note_length'],
      dtype='object')
